In [1]:
!pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.3 MB/s eta 0:00:00


In [1]:
#!pip install gensim

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import backtrader as bt
#from gensim.models import KeyedVectors
#from gensim.matutils import unitvec
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.preprocessing import StandardScaler
from collections import Counter, defaultdict
from scipy.linalg import eigh
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import os

In [63]:
# Strategy parameters
s_ticker = "AAPL"
s_start_date = "1990-01-01"
s_end_date = "2025-04-08"
i_start_capital = 1
d_trader_marging = 0.0005
i_test_trend_size = 250
i_window_size = 4
i_predict_amount = 1
i_buffer_size = 9
i_ranges_amount = 5
i_amount_excluded = 6
i_polinom_order_model = 1
b_is_using_const_model = True
d_rmse_koef = 3.0
d_probably_min = 1e-10
i_density_amount = 10
d_order_softmax_out = 3
b_is_short_enabled = False
d_zero_zone_koef = 0.01
d_trend_koef_up = 0.001
d_trend_koef_dn = 0.001
d_dither_koef = 0.7

In [4]:
# Загрузка данных из файла
def load_data_from_file(file_path):
  try:
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    return data
  except FileNotFoundError:
      print(f"Файл '{file_path}' не найден.")
      return None

# Загрузка данных из yaho finance
def load_data_from_yf(ticker, timeframe='1d'):
  df = yf.download(ticker, interval=timeframe)
  df = df.reset_index()
  if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.droplevel(level=1)
  df['Date'] = pd.to_datetime(df['Date'])
  df.set_index('Date', inplace=True)
  return df

In [6]:
# Загрузка данных из файла
file_path = 'SBER_231007.csv'  # Замените на путь к вашему файлу
data = load_data_from_file(file_path)
# Load data using yfinance
# data = yf.download(s_ticker, start=s_start_date, end=s_end_date)
#data = load_data_from_yf(s_ticker)
if data.empty:
    raise ValueError("No data downloaded for the given ticker and date range")

In [7]:
# Добавление новых признаков
data['Mean'] = (data['Open'] + data['High'] + data['Low'] + data['Close']) / 4  # Средняя цена
data['Value'] = data['Volume'] * data['Mean']  # Объём в деньгах
# Prepare trend data
p_trend_mean = data['Mean'].values
p_trend_open = data['Open'].values
p_trend_hi = data['High'].values
p_trend_lo = data['Low'].values
p_trend_close = data['Close'].values
p_trend_volume = data['Volume'].values
p_trend_value = data['Value'].values

In [8]:
# Length of trend for training
i_prep_trend_length = max(len(p_trend_mean) - i_test_trend_size, 0)

In [9]:
# Normalization function
def norming_e(p_trend, i_target_position, i_vector_size, i_samples_pos, i_amount_samples, b_is_minus_offset):
    p_result_vector = np.zeros(i_vector_size)
    d_e = np.mean(p_trend[i_target_position - i_vector_size + i_samples_pos :
                          i_target_position - i_vector_size + i_samples_pos + i_amount_samples])
    d_offset = 1 if b_is_minus_offset else 0
    p_result_vector = (p_trend[i_target_position - i_vector_size : i_target_position] / d_e) - d_offset
    return p_result_vector

In [10]:
# Calculate dot product of two vectors
def get_multip(vector1: np.ndarray, vector2: np.ndarray, i_vector_size: int) -> float:
    return np.dot(vector1[-i_vector_size:], vector2[-i_vector_size:])

In [11]:
# Calculate covariance matrix
def kovariation_matrix(trends: List[np.ndarray], matrix_size: int) -> np.ndarray:
    kov = np.zeros((matrix_size, matrix_size))
    e = np.zeros(matrix_size)
    i_trend_cols = 0

    for d_line in trends:
        if d_line is None:
            continue

        i_trend_cols += 1
        for i1 in range(matrix_size):
            d_tmp1 = d_line[i1]
            e[i1] += d_tmp1
            for i2 in range(i1, matrix_size):
                d_tmp2 = d_line[i2]
                kov[i1, i2] += (d_tmp1 * d_tmp2)

    e /= i_trend_cols
    for i1 in range(matrix_size):
        d_tmp1 = e[i1]
        for i2 in range(i1, matrix_size):
            d_tmp2 = e[i2]
            kov[i1, i2] = ((kov[i1, i2] / i_trend_cols) - (d_tmp1 * d_tmp2))
            kov[i2, i1] = kov[i1, i2]

    return kov

In [12]:
# Find eigenvectors of symmetric matrix
def get_optim_matrix_by_kov(kov: np.ndarray, i_matrix_size: int) -> Tuple[np.ndarray, np.ndarray]:
    # Get sum of diagonal elements of covariance matrix
    d_norma_vals = np.trace(kov)
    if d_norma_vals == 0:
        # Covariance matrix is degenerate
        return None, None

    # Find eigenmatrix
    try:
        params, v = eigh(kov)
    except Exception as e:
        print(f"Error finding eigenvectors: {e}")
        return None, None

    # Check and normalize eigenvalues of covariance matrix
    params /= d_norma_vals
    if np.sum(params) == 0:
        return None, None

    # Copy values to matrix
    res_matrix = np.zeros((i_matrix_size, i_matrix_size))
    for w in range(i_matrix_size):
        i_multi = 1 if v[0, w] > 0 else -1
        for i in range(i_matrix_size):
            res_matrix[w, i] = i_multi * v[i, w]

    # Sort matrix rows in descending order of eigenvalues
    sorted_indices = np.argsort(-np.abs(params))
    params = params[sorted_indices]
    res_matrix = res_matrix[sorted_indices]

    return res_matrix, params

In [13]:
# Find optimal orthogonal matrix composed of eigenvectors
def get_optim_matrix(trends: List[np.ndarray], i_matrix_size: Optional[int] = None) -> Tuple[np.ndarray, np.ndarray]:
    if i_matrix_size is None:
        i_matrix_size = len(trends[0])

    # Find covariance matrix
    kov = kovariation_matrix(trends, i_matrix_size)

    # Find eigenvectors of symmetric matrix
    return get_optim_matrix_by_kov(kov, i_matrix_size)

In [14]:
# Class representing probability density function of random variable
class DensityProbFunc:

    class DataCont:
        def __init__(self):
            self.count = 0
            self.avg = 0
            self.min = float('inf')
            self.max = float('-inf')
            self.value_p = 0

    def __init__(self, p_trend: np.ndarray, i_max_amount_distribution_points: int = 100,
                 d_min_value: Optional[float] = None, d_max_value: Optional[float] = None):
        self.d_area_min = d_min_value if d_min_value is not None else np.min(p_trend)
        self.d_area_max = d_max_value if d_max_value is not None else np.max(p_trend)
        self.m_distribution = {}

        self.create_density(p_trend, i_max_amount_distribution_points, d_min_value, d_max_value)

    def create_density(self, p_trend: np.ndarray, i_max_amount_distribution_points: int = 100,
                      d_min_value: Optional[float] = None, d_max_value: Optional[float] = None):
        if i_max_amount_distribution_points <= 0:
            raise ValueError("Invalid number of distribution function ranges")

        self.m_distribution.clear()
        self.d_area_min = d_min_value if d_min_value is not None else np.min(p_trend)
        self.d_area_max = d_max_value if d_max_value is not None else np.max(p_trend)

        d_inc_val = (self.d_area_max - self.d_area_min) / i_max_amount_distribution_points
        if d_inc_val == 0:
            self.m_distribution[0] = self.DataCont()
            self.m_distribution[0].count = len(p_trend)
            self.m_distribution[0].avg = self.d_area_min
            self.m_distribution[0].min = self.d_area_min
            self.m_distribution[0].max = self.d_area_max
            self.m_distribution[0].value_p = 1
            return

        for i_index in range(i_max_amount_distribution_points):
            p_data_cont = self.DataCont()
            p_data_cont.min = self.d_area_min + (d_inc_val * i_index)
            p_data_cont.max = p_data_cont.min + d_inc_val
            self.m_distribution[i_index] = p_data_cont

        for d_value in p_trend:
            i_index = int((d_value - self.d_area_min) / d_inc_val)
            if i_index < 0 or i_index >= i_max_amount_distribution_points:
                continue
            p_data_cont = self.m_distribution[i_index]
            p_data_cont.avg += d_value
            p_data_cont.count += 1

        # Set values for non-zero clusters
        for i_index in range(i_max_amount_distribution_points):
            p_data_cont = self.m_distribution.get(i_index)
            if p_data_cont is None or p_data_cont.count == 0:
                continue
            p_data_cont.avg /= p_data_cont.count
            p_data_cont.value_p = p_data_cont.count / len(p_trend)

    # Get probability density value
    def get_density_value(self, d_trend_val: float) -> float:
        if not self.m_distribution:
            return 0

        d_inc_val = (self.d_area_max - self.d_area_min) / len(self.m_distribution)
        if d_inc_val == 0:
            d_inc_val = float('inf')

        i_index = int((d_trend_val - self.d_area_min) / d_inc_val)
        if i_index < 0 or i_index >= len(self.m_distribution):
            return 0

        return self.m_distribution.get(i_index, self.DataCont()).value_p

In [15]:
# Building a plane based on target and feature data
class GMDH:

    def __init__(self):
        self.m_mgua_k = None
        self.m_mgua_max_context = 0
        self.m_mgua_max_order = 0
        self.m_mgua_total_arg_numb = 0
        self.m_mgua_is_const_used = True

    # Create model from linear vector
    def create_from_linear_vector(self, p_vector_data: np.ndarray, i_max_context: int = 0,
                                is_const_used: bool = False, d_const_val: float = 0):
        # Save context size
        if i_max_context <= 0:
            i_max_context = len(p_vector_data)
        self.m_mgua_max_context = i_max_context

        # Save model order
        self.m_mgua_max_order = 1

        # Save use of constant offset
        self.m_mgua_is_const_used = is_const_used

        # Determine number of model arguments
        i_offset = 1 if self.m_mgua_is_const_used else 0
        self.m_mgua_total_arg_numb = self.m_mgua_max_context + i_offset

        # Create vector and copy data to it
        self.m_mgua_k = np.zeros(self.m_mgua_total_arg_numb)
        for i in range(self.m_mgua_max_context):
            self.m_mgua_k[i + i_offset] = p_vector_data[i]
        if self.m_mgua_is_const_used:
            self.m_mgua_k[0] = d_const_val

    # Group method of data handling with order up to maximum
    def create_model(self, vals: np.ndarray, trend: np.ndarray, i_max_context: int,
                    i_max_order: int, i_target_position: int = 0, i_start_position: int = 0,
                    is_const_used: bool = True) -> bool:
        # Save context size
        self.m_mgua_max_context = i_max_context
        if self.m_mgua_max_context < 2:
            return False

        # Save model order
        self.m_mgua_max_order = i_max_order
        if self.m_mgua_max_order < 1:
            return False

        # Save use of constant offset
        self.m_mgua_is_const_used = is_const_used

        # If end index is not set, take whole sample length
        if i_target_position <= 0:
            i_target_position = len(vals)

        # Determine number of model arguments
        self.m_mgua_total_arg_numb = self.get_param_numb(i_max_context, self.m_mgua_max_order, is_const_used)

        # Create buffers if they haven't been created or their size is less than needed
        if self.m_mgua_k is None or len(self.m_mgua_k) < self.m_mgua_total_arg_numb:
            self.m_mgua_k = np.zeros(self.m_mgua_total_arg_numb)

        # Initialize buffers
        a_matrix = np.zeros((self.m_mgua_total_arg_numb, self.m_mgua_total_arg_numb))
        b_vector = np.zeros(self.m_mgua_total_arg_numb)

        # For all input vectors
        for t in range(i_start_position, i_target_position):
            # Set input parameter values to vector K
            k_vector = np.zeros(self.m_mgua_total_arg_numb)
            self.set_param_buf(self.m_mgua_max_context, self.m_mgua_max_order, vals[t], k_vector, is_const_used)

            # Value from training vector
            d_curr_value = trend[t]

            # Update statistics
            for i0 in range(self.m_mgua_total_arg_numb):
                k_i0 = k_vector[i0]
                for i1 in range(i0 + 1):
                    a_matrix[i0, i1] += (k_i0 * k_vector[i1])
                b_vector[i0] += (k_i0 * d_curr_value)

        # Find model coefficients vector using least squares
        try:
            self.m_mgua_k = np.linalg.lstsq(a_matrix, b_vector, rcond=None)[0]
        except np.linalg.LinAlgError:
            return False

        return (not np.any(np.isnan(self.m_mgua_k))) and (not np.any(np.isinf(self.m_mgua_k))) and (np.sum(np.abs(self.m_mgua_k)) != 0)

    #  Calculate predicted value using existing model
    def calculate_prognos(self, sample: np.ndarray) -> float:
        if self.m_mgua_max_order < 1:
            return 0

        # Calculate predicted value using found model
        k_vector = np.zeros(self.m_mgua_total_arg_numb)
        self.set_param_buf(self.m_mgua_max_context, self.m_mgua_max_order, sample, k_vector, self.m_mgua_is_const_used)
        return np.dot(k_vector, self.m_mgua_k)

    # Return number of variables for given context size
    @staticmethod
    def get_param_numb(i_max_context: int, i_max_order: int, is_const_used: bool) -> int:
        i_count = 1
        i_total = 1 if is_const_used else 0  # Account for constant component

        for i_curr_order in range(1, i_max_order + 1):
            j = i_max_context + i_curr_order - 1
            i_count = i_count * j // i_curr_order
            i_total += i_count

        return i_total

    # Set input vector of length i_max_context to vector that is argument for GMDH method
    @staticmethod
    def set_param_buf(i_max_context: int, i_max_order: int, inp_buf: np.ndarray,
                     out_buf: np.ndarray, is_const_used: bool):
        i_count = 0

        for i_curr_order in range(0 if is_const_used else 1, i_max_order + 1):
            i_count = GMDH.order_param_create(i_count, i_curr_order, 0, i_max_context, 1, inp_buf, out_buf)

    # Recursive function for setting buffer
    @staticmethod
    def order_param_create(i_count: int, i_level: int, i_context: int, i_max_context: int,
                          d_upper: float, inp_buf: np.ndarray, out_buf: np.ndarray) -> int:
        if i_level > 0:
            # This function is called recursively
            for i_context in range(i_context, i_max_context):
                i_count = GMDH.order_param_create(i_count, i_level - 1, i_context, i_max_context,
                                                d_upper * inp_buf[i_context], inp_buf, out_buf)
        else:
            out_buf[i_count] = d_upper
            i_count += 1

        return i_count

In [16]:
# Forming list of normalized trend changes
p_delta_filter_trends = {}
for tt in range(i_window_size, i_prep_trend_length - i_predict_amount + 1):
    p_norm_trend = norming_e(p_trend_mean, tt + i_predict_amount, i_window_size + i_predict_amount,
                            0, i_window_size, True)
    p_delta_filter_trends[tt] = p_norm_trend[i_window_size] - p_norm_trend[i_window_size - 1]

In [17]:
# Forming list of normalized trends for all positions
p_buffer_trends = {}
for tt in range(i_buffer_size, len(p_trend_mean) + 1):
    p_buffer = np.concatenate([
        norming_e(p_trend_open, tt, i_buffer_size, 0, i_buffer_size, True),
        norming_e(p_trend_lo, tt, i_buffer_size, 0, i_buffer_size, True),
        norming_e(p_trend_hi, tt, i_buffer_size, 0, i_buffer_size, True),
        norming_e(p_trend_close, tt, i_buffer_size, 0, i_buffer_size, True),
        norming_e(p_trend_volume, tt, i_buffer_size, 0, i_buffer_size, True),
        norming_e(p_trend_value, tt, i_buffer_size, 0, i_buffer_size, True)
    ])
    if not (np.isinf(p_buffer).any() and not np.isnan(p_buffer).any()):
        p_buffer_trends[tt] = p_buffer

In [18]:
# Forming trend arrays grouped by change value
d_delta = (len(p_delta_filter_trends) - (i_buffer_size - i_window_size if i_buffer_size > i_window_size else 0)) / i_ranges_amount
if d_delta < 2:
    raise ValueError("Minimum 2 elements per range required")

i_counter = 0
i_zero_index = 0
p_ranges_trends = [set() for _ in range(i_ranges_amount)]

for tt, val in sorted(p_delta_filter_trends.items(), key=lambda x: x[1]):
    if tt not in p_buffer_trends:
        continue
    i_index = min(int(i_counter / d_delta), i_ranges_amount - 1)
    p_ranges_trends[i_index].add(tt)
    if val < 0 and i_zero_index < i_index:
        i_zero_index = i_index
    i_counter += 1

# Refine zero position
i_pos_count = sum(1 for tt in p_ranges_trends[i_zero_index] if p_delta_filter_trends[tt] > 0)
i_neg_count = sum(1 for tt in p_ranges_trends[i_zero_index] if p_delta_filter_trends[tt] < 0)
d_zero_value = i_neg_count / (i_neg_count + i_pos_count) if (i_neg_count + i_pos_count) > 0 else 0.5

In [19]:
# Orthogonal matrices for ranges
p_ort_matrix = [None] * i_ranges_amount
p_eigen_vals = [None] * i_ranges_amount
p_range_models = [None] * i_ranges_amount
p_models_distrib_funcs = [None] * i_ranges_amount

for i_index1 in range(i_ranges_amount):
    # Form orthogonal matrix
    p_curr_data_set_src = [p_buffer_trends[tt] for tt in p_ranges_trends[i_index1]]
    if not p_curr_data_set_src:
        continue

    # Calculate optimal orthogonal matrix
    p_ort_matrix[i_index1], p_eigen_vals[i_index1] = get_optim_matrix(p_curr_data_set_src)

    # Recalculate for entire buffer
    p_buffer_trends2 = {}
    for tt, values in p_buffer_trends.items():
        p_matrix_spektr = np.zeros(len(values) - i_amount_excluded)
        for w in range(len(p_matrix_spektr)):
            d_val = np.dot(p_ort_matrix[i_index1][w], values[:len(p_ort_matrix[i_index1][w])])
            p_matrix_spektr[w] = d_val
        p_buffer_trends2[tt] = p_matrix_spektr

    # Add to list of unit orthogonal functions planes
    p_models_curr = []
    p_base_vector = np.zeros(len(p_buffer_trends2[next(iter(p_buffer_trends2))]))

    for w in range(len(p_base_vector)):
        p_base_vector[w] = 1
        p_ort_matrix_model = GMDH()
        p_ort_matrix_model.create_from_linear_vector(p_base_vector, len(p_base_vector))
        p_models_curr.append(p_ort_matrix_model)
        p_base_vector[w] = 0

    # Forming mutual range boundary models
    p_curr_data_set1 = [p_buffer_trends2[tt] for tt in p_ranges_trends[i_index1]]
    if not p_curr_data_set1:
        continue

    for i_index2 in range(i_ranges_amount):
        if i_index2 == i_index1:
            continue

        p_curr_data_set2 = [p_buffer_trends2[tt] for tt in p_ranges_trends[i_index2]]
        if not p_curr_data_set2:
            continue

        # Calculate GMDH model
        p_curr_model = GMDH()
        p_src_list = np.array(p_curr_data_set1 + p_curr_data_set2)
        p_dst_list = np.array([1.0] * len(p_curr_data_set1) + [-1.0] * len(p_curr_data_set2))
        if p_curr_model.create_model(p_src_list, p_dst_list, len(p_src_list[0]), i_polinom_order_model,
                                  len(p_src_list), 0, b_is_using_const_model):
            p_models_curr.append(p_curr_model)

    p_range_models[i_index1] = p_models_curr

    # Calculate probability densities for mutual GMDH range boundary models
    p_curr_distrib_funcs = []
    for w in range(len(p_models_curr)):
        p_matrix_spektr = np.array([p_models_curr[w].calculate_prognos(sample) for sample in p_curr_data_set1])
        d_e = np.mean(p_matrix_spektr)
        d_ee = np.mean(p_matrix_spektr ** 2)
        d_sigma = np.sqrt(d_ee - (d_e * d_e))
        p_min_value = max(d_e - (d_rmse_koef * d_sigma), np.min(p_matrix_spektr))
        p_max_value = min(d_e + (d_rmse_koef * d_sigma), np.max(p_matrix_spektr))
        p_curr_distrib_funcs.append(DensityProbFunc(p_matrix_spektr, i_density_amount,
                                                    p_min_value, p_max_value))
    p_models_distrib_funcs[i_index1] = p_curr_distrib_funcs

In [64]:
# Recalculation considering distribution function
p_buffer_trends_out = {}
for tt, values in p_buffer_trends.items():
    d_sum_vals = 0
    p_result = np.zeros(i_ranges_amount)

    for i_index in range(i_ranges_amount):
        d_val = 0
        # Recalculate buffer by optimal orthogonal matrix for range
        if p_ort_matrix[i_index] is None:
            continue

        p_buf_curr = np.zeros(len(values) - i_amount_excluded)
        for w in range(len(p_buf_curr)):
            p_buf_curr[w] = np.dot(p_ort_matrix[i_index][w], values[:len(p_ort_matrix[i_index][w])])

        # Calculate probabilities of divergence by mutual range boundary models vectors
        p_models_curr = p_range_models[i_index]
        if p_models_curr is not None:
            p_curr_distrib_funcs = p_models_distrib_funcs[i_index]
            for w in range(len(p_curr_distrib_funcs)):
                d_sp = p_models_curr[w].calculate_prognos(p_buf_curr)
                d_sp = p_curr_distrib_funcs[w].get_density_value(d_sp)
                if d_sp == 0:
                    d_sp = d_probably_min
                d_val += np.log(d_sp)
        #d_val = np.exp(d_val)
        d_val = d_order_softmax_out ** d_val  # Softmax
        d_sum_vals += d_val
        p_result[i_index] = d_val

    p_result /= d_sum_vals
    p_buffer_trends_out[tt] = p_result

In [65]:
# Recalculation of output buffer trends
p_out_result_trends = {}
for tt, values in p_buffer_trends_out.items():
    d_probs0 = np.sum(values[:i_zero_index]) + d_zero_value * values[i_zero_index]
    d_probs1 = np.sum(values[i_zero_index+1:]) + (1.0 - d_zero_value) * values[i_zero_index]
    p_out_result_trends[tt] = (d_probs1 - d_probs0) / (d_probs0 + d_probs1)

In [66]:
count_less_than_0 = sum(1 for value in p_out_result_trends.values() if value < 0)
count_great_than_0 = sum(1 for value in p_out_result_trends.values() if value > 0)
count_not_value = sum(1 for value in p_out_result_trends.values() if np.isnan(value) or np.isinf(value))
print(f"Количество элементов меньше 0: {count_less_than_0}")
print(f"Количество элементов больше 0: {count_great_than_0}")
print(f"Количество элементов не чисел: {count_not_value}")

Количество элементов меньше 0: 2936
Количество элементов больше 0: 3123
Количество элементов не чисел: 0


In [36]:
#len(p_models_distrib_funcs[4])
#len(p_out_result_trends)

52

In [38]:
# ************* Trading simulation on historical data *************
# Создание стратегии для backtrader на основе Ensemble
class SimulationStrategyEnsemble(bt.Strategy):
    def __init__(self):
        self.prediction_index = 0  # Индекс для отслеживания текущего предсказания

    def next(self):
        if self.prediction_index < len(p_buffer_trends_out):
            prediction = p_buffer_trends_out[self.prediction_index]
            if prediction > 0:  # Если предсказание роста цены
                if not self.position:
                    self.buy()  # Открываем длинную позицию
            elif prediction < 0:  # Если предсказание падения цены
                if self.position:
                    self.sell()  # Закрываем позицию
            self.prediction_index += 1

In [50]:
# Backtrader strategy implementation
class BacktestStrategy(bt.Strategy):
    params = (
        ('out_result_trends', p_out_result_trends),
        ('prep_trend_length', i_prep_trend_length),
        ('zero_zone_koef', d_zero_zone_koef),
        ('trend_koef_up', d_trend_koef_up),
        ('trend_koef_dn', d_trend_koef_dn),
        ('dither_koef', d_dither_koef),
        ('is_short_enabled', b_is_short_enabled)
    )

    def __init__(self):
        self.extreme_price_min = self.data.close[0]
        self.extreme_price_max = self.data.close[0]
        self.avg_index_prev = 0
        self.order = None
        self.money_count_trend = []

    def next(self):
        current_idx = len(self.data) - 1
        t = current_idx + self.p.prep_trend_length

        if t not in self.p.out_result_trends:
            return

        # Get complex indicator value for this position
        d_avg_index = self.p.out_result_trends[t]

        # Exponential smoothing
        if self.avg_index_prev != 0:
            d_avg_index = (self.p.dither_koef * d_avg_index) + (1 - self.p.dither_koef) * self.avg_index_prev
        self.avg_index_prev = d_avg_index

        # Check if in cash (no stocks)
        if abs(self.position.size) <= 1e-10:
            # Check for Long entry
            d_th_price = self.extreme_price_min * (1.0 + self.p.trend_koef_up)
            if (d_th_price <= self.data.high[0]) and (d_avg_index > self.p.zero_zone_koef):
                if d_th_price < self.data.open[0]:
                    d_th_price = self.data.open[0]
                # Need to buy
                self.order = self.buy(price=d_th_price)
                self.extreme_price_max = self.extreme_price_min = d_th_price
            elif self.p.is_short_enabled:
                # Check for Short entry
                d_th_price = self.extreme_price_max * (1.0 - self.p.trend_koef_dn)
                if (self.data.low[0] <= d_th_price) and (d_avg_index < -self.p.zero_zone_koef):
                    if d_th_price > self.data.open[0]:
                        d_th_price = self.data.open[0]
                    # Need to sell
                    self.order = self.sell(price=d_th_price)
                    self.extreme_price_min = self.extreme_price_max = d_th_price

        # In Long position
        elif self.position.size > 0:
            # Check for exit to Cash
            d_th_price = self.extreme_price_max * (1.0 - self.p.trend_koef_dn)
            if (self.data.low[0] <= d_th_price) and (d_avg_index < -self.p.zero_zone_koef):
                if d_th_price > self.data.open[0]:
                    d_th_price = self.data.open[0]
                # Need to sell
                self.order = self.close(price=d_th_price)
                self.extreme_price_min = self.extreme_price_max = d_th_price

        # In Short position
        elif self.position.size < 0:
            # Check for exit to Cash
            d_th_price = self.extreme_price_min * (1.0 + self.p.trend_koef_up)
            if (d_th_price <= self.data.high[0]) and (d_avg_index > self.p.zero_zone_koef):
                if d_th_price < self.data.open[0]:
                    d_th_price = self.data.open[0]
                self.order = self.close(price=d_th_price)
                self.extreme_price_max = self.extreme_price_min = d_th_price

        # Search for minimum for position entry
        self.extreme_price_min = min(self.extreme_price_min, self.data.low[0])
        # Search for maximum for position entry
        self.extreme_price_max = max(self.extreme_price_max, self.data.high[0])

        # Save portfolio value
        self.money_count_trend.append(self.broker.getvalue())

    def stop(self):
        # Final portfolio value
        self.money_count_trend.append(self.broker.getvalue())

        # Output results
        print(f"Maximum result value = {self.broker.getvalue()}")
        print(f"Optimal data window size = {i_window_size}")
        print(f"Optimal prediction count = {i_predict_amount}")
        print(f"Optimal buffer window size = {i_buffer_size}")
        print(f"Optimal range count = {i_ranges_amount}")
        print(f"Optimal probability density intervals = {i_density_amount}")
        print(f"Optimal smoothing coefficient = {d_dither_koef}")
        print(f"Optimal Up trade cutoff coefficient = {d_trend_koef_up}")
        print(f"Optimal Dn trade cutoff coefficient = {d_trend_koef_dn}")
        print(f"Optimal 'zero zone' coefficient = {d_zero_zone_koef}")

        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(self.money_count_trend)
        plt.title("Portfolio Value Over Time")
        plt.xlabel("Time")
        plt.ylabel("Portfolio Value")

        # Save plot
        output_dir = "results"
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir,
            f"{s_ticker}_Result={self.broker.getvalue():.5f}_WndSize={i_window_size}_Predict={i_predict_amount}.png")
        plt.savefig(output_file)
        plt.close()

In [39]:
# Create cerebro engine
cerebro = bt.Cerebro()

# Add strategy
#cerebro.addstrategy(BacktestStrategy)
cerebro.addstrategy(SimulationStrategyEnsemble)

# Prepare data feed
test_data = data.iloc[-i_test_trend_size:]  # Выбираем тестовую часть данных
test_data = bt.feeds.PandasData(dataname=test_data)
cerebro.adddata(test_data)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe", timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')

# Set initial capital
cerebro.broker.setcash(i_start_capital)

# Set commission
cerebro.broker.setcommission(commission=d_trader_marging)

# Run backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot results if needed
# cerebro.plot()

Starting Portfolio Value: 1.00


/usr/local/lib/python3.11/dist-packages/backtrader/feeds/pandafeed.py:268: UserWarning: Discarding nonzero nanoseconds in conversion.
  dt = tstamp.to_pydatetime()


KeyError: 0